In [1]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from summarytools import dfSummary

# Define connection parameters
server = 'QS-LOGU\SQLEXPRESS'       # e.g., localhost or ServerName\InstanceName
database = 'MathsLearning'   # e.g., MathE dataset
username = ''      # Leave blank if using Windows Authentication
password = ''     # Leave blank if using Windows Authentication

# Create connection string
if username and password:
    connection_string = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server"
else:
    connection_string = f"mssql+pyodbc://@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server&Trusted_Connection=yes"

# Create SQLAlchemy engine
engine = create_engine(connection_string)

# Query the data
query = "SELECT * FROM [dbo].[maths_dataset];"  
df = pd.read_sql(query, engine)

# Display the data
print(df.head())

#Close the engine
engine.dispose()


   Student_ID Student_Country  Question_ID  Type_of_Answer Question_Level  \
0         647         Ireland           77               0          Basic   
1          41        Portugal           77               1          Basic   
2         340        Portugal           77               1          Basic   
3         641           Italy           77               0          Basic   
4         669        Portugal           77               1          Basic   

        Topic    Subtopic                                           Keywords  
0  Statistics  Statistics  Stem and Leaf diagram,Relative frequency,Sampl...  
1  Statistics  Statistics  Stem and Leaf diagram,Relative frequency,Sampl...  
2  Statistics  Statistics  Stem and Leaf diagram,Relative frequency,Sampl...  
3  Statistics  Statistics  Stem and Leaf diagram,Relative frequency,Sampl...  
4  Statistics  Statistics  Stem and Leaf diagram,Relative frequency,Sampl...  


In [2]:
dfSummary(data=df, is_collapsible=False)

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,Student_ID[int64],Mean (sd) : 775.4 (460.6)min < med < max:26.0 < 885.0 < 1565.0IQR (CV) : 839.0 (1.7),372 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8g+/7EAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACsklEQVR4nO3cz4raQBzA8d/UVaJCggh68wUKPXj0IfqwfYQefALfwJsgIWK6wZikh9JF6FbNjNH5Dd/PeTObw5dO50/WNE0jgO++vPoFgHu83foBY8xQRAaW45+apnm3fBb4cDVUY8xwOp1+j+N4YjP44XBIjTE/iBWubv2LOojjeLJard6TJCnaDJxlWbReryf7/X4gIoQKJzenfhGRJEmK2Wz2y2L8ocUzwD9YTEEFQoUKhAoVCBUqECpUIFSoQKhQgVChwl0b/tAj1LsZhBqQkO9mEGpYgr2bQaiecZy647qu+0mSpKHdzSBUj7hO3WVZRlVVfS2K4qeI2ITqLUL1i/XULSKy3W4nm81mdD6fe1283CsRqodsr1Wmaert1O2KfVSoQKhQgVChAqFCBUKFCoQKFQgVKhAqVCBUqECoUMHrI9RQLwGjPW9DDfkSMNrzNlQJ+BIw2vM5VBHhD7ThDxZTUIFQoYL3Uz+ep6qqvojExhjbITrbaSFUiIhInuf9siy/LRaLXq/Xa/0ZjEi3Oy2E2gGH/d+4ruv+o9/nHqfT6S2KotFyuSzm83na9vmud1qCDfVV05jL/q8PX5GOx2PbXRaRDndaggz1xdOY9f5vyF+RugoyVB+mMZv935C/InUVZKh/+TqNoT32UaECoUIFQoUKhAoVCBUqECpUCHp7Cs/leBp49SSQUPEQrqeBt04CCRUP4XIaeM9JIKHioRxOA6+eBLKYggqEChWY+v/DYQX7ssvPISPUT7isYH24/BwiQv2EywqWy8/dINQrbFawXH7uBospqECoUKHTqd/x7JfVMz50Fqrr2S+rZ1zqLFTXL0FZPeNS56t+27NfVs+4xGIKKhAqVCBUqHDX/1GzLIvaDnw8HiMRkTzPo91uN3rm81p/t9b3dn3+nr5+A5EGvMXYDrlqAAAAAElFTkSuQmCC"">",0(0.0%)
2,Student_Country[object],1. Portugal2. Lithuania3. Italy4. Slovenia5. Ireland6. Russian Federation7. Romania8. Spain,"5,495 (57.6%)1,443 (15.1%)1,358 (14.2%)755 (7.9%)300 (3.1%)107 (1.1%)60 (0.6%)28 (0.3%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAC4CAYAAAD0WZ4UAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8g+/7EAAAACXBIWXMAAA9hAAAPYQGoP6dpAAADaUlEQVR4nO3dQU7bUBiF0d9RRJuJoygSS2hnLIBFsNguIgtg2D2gCA8aYJIOSimEMEjgXVfhnBU8VV9tbPlCt91uCxImYx+Az0NsxIiNGLERIzZixEZMV1Wzqjob+yAn7mG73W7GPsTYpsvl8qrv+8XYBzllwzCsu6778dmDm/Z9v7i8vNzM5/O7sQ9zim5vb7+uVqvFzc3NWVV97tiqqubz+d35+fmvsQ9zwmZjH+B/4AGBGLERIzZixEaM2IgRGzFiI0ZsxIiNGLERIzZixEaM2IiZVv35DGbsg5wq/7b/TIdhWK9Wq0X5DKaZYRjWVfUw9jnG5rPwDJ+FV1Xn1y+Q0vrK5n80T5oOXgw9eK7Z4MXQg12tBy+ecHnipS4xYiNGbMSIjRixESM2YsRGjNiIERsxYiNGbMSIjRixEdNs8GLowa6mgxdDD57zWTgxBi/EeEAgpuVt1C2UF5qtqyyr2NVkXWVZxT4t11WWVbzgAYEYsREjNmLERozYiBEbMWIjRmzEiI0YsREjNmLERozYiGmyrrKsYp9m6yrLKnb5LJwY6ypijrmyuWJxlIMHL4YsHOugwYshC+9xzODFkIWjeKlLjNiIERsxYiNGbMSIjRixESM2YsRGjNiIERsxYiNGbMQcNHgxZOE9Dh68GLJwLJ+FE2PwQowHBGLeuo26VfLh9q6rLKho4dW6yoKKVt5aV1lQ8eE8IBAjNmLERozYiBEbMWIjRmzEiI0YsREjNmLERozYiBEbMa/WVRZUtLJ3XWVBRQs+CyfGuoqY51c2VzOaehq8GLnQ2qTv+8XFxUU9Lqxa/ClIqKrH92yz2ex+7INw+rzUJUZsxIiNGLERIzZixEaM2IgRGzFiI0ZsxIiNGLERIzZixEbMpKpqs9l8GfsgnL7JMAzr6+triyqas0EgxrqKmL9XtnJVo7Xpcrm8qqqyrKK1Sd/3C8sqErxnI0ZsxIiNGLERIzZixEaM2IgRGzFiI0ZsxIiNGLERIzZixEaM2IiZPg5dqoxdaMxn4cQYvBDjZzZiJl3Xfe+6zt8YpblJVX2rKr9+gebcRokRGzFiI0ZsxIiNGLERIzZixEaM2IgRGzFiI0ZsxIiNGLERIzZixEbMpKp+VtX92Afh9Bm8EOM2SozYiBEbMWIjRmzEiI2Y35IST3t9PRJOAAAAAElFTkSuQmCC"">",0(0.0%)
3,Question_ID[int64],Mean (sd) : 478.9 (249.2)min < med < max:77.0 < 428.0 < 1549.0IQR (CV) : 248.0 (1.9),833 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8g+/7EAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACsElEQVR4nO3cT24aMRiH4c+FoIFIgxAV7LhApS5Y5hA9bI7QSJwgN8gOaYQ8gkKBYbpqV2kS7JkyP/d99k68eGXmn+3qujag6z7degLAR/RvPYG3OOeGZjYIHH6s63rf5HxwO50N1Tk3nE6n3/I8n4SML8ty45x7JNY0dDZUMxvkeT55eHjYj8fjwzUDvffZarWaFEUxMDNCTUCXQzUzs/F4fJjNZj8Chg4bnwxuhpspSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSGj1pJTIQ87yy+Vy1+R8oKu1UGMPOTudTllVVV8Oh8N3Mws50gcJaXNFDT7kzMzs5eVl8vz8PDqfz702JgctrR+SFnrI2Waz4ZAz/MHNFCQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiS8+z1qxHYStpKgMW+GGrOdhK0kaNJ7K2rwdhL1rSSRGxOPdV3vm5zP/+5DW1FCtpMobyWJ3ZhYluXGOfdIrM1pfc+UqOBfEu99tlqtJkVRDMyMUBtCqG8I3ZhoZrK/Jl2VbKhVVd2ZWe6cCxnOE4uOSTLU3W53dzqdvi4Wi16v17v6TAGeWHRPkqEej8d+lmWj5XJ5mM/nm2

a. -Are there differences in performance (Type of Answer) between students from different countries (Student Country)?

In [3]:
#Differences in performance between students from diffrent countries 
performance = (
    df.groupby('Student_Country')['Type_of_Answer']
    .agg(total_correct='sum', total_answers='count')
)
performance['percentage_correct'] = (performance['total_correct'] / performance['total_answers']) * 100
print(performance)
performance = performance.reset_index()

fig = px.bar(performance, x='Student_Country', y='percentage_correct', color_continuous_scale="Viridis")
fig.update_layout(
    title='Differences in Perfomance Between Students of Different Countries',
    title_font_size = 22,
    xaxis_title = 'Student Country',
    yaxis_title = 'Percentage Difference'
)
fig.show()


                    total_correct  total_answers  percentage_correct
Student_Country                                                     
Ireland                       138            300           46.000000
Italy                         606           1358           44.624448
Lithuania                     629           1443           43.589744
Portugal                     2494           5495           45.386715
Romania                        35             60           58.333333
Russian Federation             37            107           34.579439
Slovenia                      519            755           68.741722
Spain                          12             28           42.857143


b. Which countries have the highest percentage of correct answers (Type of Answer = 
1)?


In [4]:
#Countries with the highest percentage of correct answers
performance_by_country = (df.groupby('Student_Country')['Type_of_Answer'].agg(['sum', 'count']).reset_index())


performance_by_country['PerformancePercentage'] = (performance_by_country['sum'] * 100 / performance_by_country['count'])


performance_by_country = performance_by_country[['Student_Country', 'PerformancePercentage']]

performance_by_country = performance_by_country.sort_values(by='PerformancePercentage', ascending=False)


print(performance_by_country.head())

  Student_Country  PerformancePercentage
6        Slovenia              68.741722
4         Romania              58.333333
0         Ireland              46.000000
3        Portugal              45.386715
1           Italy              44.624448


c. How does the distribution of correct vs incorrect answers vary by country?


In [14]:
#Distribution of correct vs incorrect answers by country

answer_distribution = (
    df.groupby(["Student_Country", "Type_of_Answer"]).size().reset_index(name="Count")
)

answer_distribution["Total"] = answer_distribution.groupby("Student_Country")["Count"].transform("sum")
answer_distribution["Percentage"] = (answer_distribution["Count"] / answer_distribution["Total"]) * 100

# Plot 100% stacked bar chart
fig = px.bar(
    answer_distribution,
    x="Student_Country",
    y="Percentage",
    color="Type_of_Answer",
    text="Percentage",
    title="Answer Distribution by Country",
    #labels={"Percentage": "Percentage (%)", "Student_Country": "Country"},
    height= 600,
    width= 1000
)

# Customize bar chart
fig.update_traces(
    texttemplate="%{text:.1f}%",  # Format percentages inside bars
    textposition="inside",
)

fig.update_layout(
    yaxis_title="Distribution (%)",
    xaxis_title="Student Country",
    barmode="stack",  # 100% stacked bar
    title_font_size=20,
    legend_title_text="Answer Type",
    template="plotly_white",
)

fig.show()


d. Is there a pattern in the Type of Answer (correct or incorrect) by the Question Level?


In [48]:
questionPattern = (df.groupby(['Question_Level', 'Type_of_Answer']).size().reset_index(name= 'Count'))
questionPattern['Total'] = questionPattern.groupby('Question_Level')['Count'].transform('sum')
questionPattern['Percentage'] = (questionPattern['Count']/questionPattern['Total'])*100

fig = px.bar(
    questionPattern,
    x= 'Question_Level',
    y= 'Percentage',
    title= 'Distribution of Answers By Question Type',
    color= 'Type_of_Answer',
    text= 'Percentage',
    height= 600,
    width= 600,
)

fig.update_traces( texttemplate = '%{text:.1f}%',
                  textposition = 'inside'    
)

fig.update_layout(
    xaxis_title = 'Question Type',
    yaxis_title = 'Percentage Distribution',
    barmode = 'stack',
    legend_title = 'Answer Type'
)
fig.show()

e. What is the average success rate or different question levels (e.g., easy vs hard)?

In [36]:
#Average success rate for different question levels
averageSuccessRate = df.groupby('Question_Level')['Type_of_Answer'].mean().reset_index(name='Success Rate')
averageSuccessRate["Success Rate"] *= 100
averageSuccessRate

,Question_Level,Success Rate
0,Advanced,50.117509
1,Basic,46.111678


In [47]:

fig = px.bar(
    averageSuccessRate,
    x= 'Question_Level',
    y= 'Success Rate',
    color= 'Success Rate',
    title= 'Average Success Rate For Different Question Levels',
    text= 'Success Rate',
    height= 500,
    width= 500
)
fig.update_traces(
    texttemplate = '%{text:.1f}%',
    textposition = 'outside'
)
fig.update_layout(
    xaxis_title = 'Question Level',
    yaxis_title = 'Rate (%)',
    legend_title = 'Type of Answer'
)
fig.show()

f. Which topics are students performing the best in? (i.e., highest percentage of correct answers per Topic)

In [46]:
#Highest percentage of correct answers by topics

performanceByTopic = df.groupby('Topic')['Type_of_Answer'].agg(['sum', 'count']).reset_index()

performanceByTopic['percentage'] = (performanceByTopic['sum']/performanceByTopic['count'])*100

performanceByTopic = performanceByTopic[['Topic', 'percentage']]

performanceByTopic = performanceByTopic.sort_values('percentage')

print(performanceByTopic)

performanceByTopic_df = performanceByTopic.reset_index()

fig = px.bar(
    performanceByTopic_df,
    y='Topic',
    x='percentage',
    title='Success Rate by Topic',
    labels={'percentage': 'percentage', 'Topic': 'Topic'},
    text='percentage',
    height=600,
    width=1000,
    color='percentage',
    #color_continuous_scale='Blues'
)

fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
fig.update_layout(xaxis_title="Topic", yaxis_title="percentage", xaxis_tickangle=45)

fig.show()

                                  Topic  percentage
3                       Differentiation   34.196891
11  Real Functions of a single variable   35.365854
10                         Probability    37.500000
9                          Optimization   38.461538
8                     Numerical Methods   38.709677
6                           Integration   44.444444
1                       Complex Numbers   45.439189
4               Fundamental Mathematics   46.577017
13                           Statistics   47.941176
0                     Analytic Geometry   48.882682
7                        Linear Algebra   49.022005
2                Differential Equations   53.703704
5                          Graph Theory   58.181818
12                           Set Theory   64.285714


g. Are there specific subtopics where students struggle more often?

In [59]:
#Subtopics with hishest number of incorrect answers 

#filter data to include only 'Type_of_Answer =0'
# Group by 'Subtopic' and count the number of struggles (Type_of_Answer == 0)
struggleSubtopics = df[df['Type_of_Answer'] == 0].groupby('Subtopic').size()

# Total occurrences of each subtopic
totalSubtopics = df.groupby('Subtopic').size()

# Calculate the percentage of struggles for each subtopic
struggleSubtopics = (struggleSubtopics / totalSubtopics) * 100

# Sort by percentage in descending order
struggleSubtopics = struggleSubtopics.sort_values(ascending=False)

# Convert Series to DataFrame and reset index
struggleSubtopics = struggleSubtopics.reset_index(name='Percentage')

fig = px.bar(
    struggleSubtopics,
    x= 'Percentage',
    y= 'Subtopic',
    color= 'Subtopic',
    text= 'Percentage',
    height= 800,
    width= 1500
)
fig.update_traces(texttemplate = '%{text:.1f}%', textposition = 'outside')
fig.update_layout(
    xaxis_title = 'Incorrect (%)',
    yaxis_title = 'Subtopics',

)
fig.show()

c:\Users\QS LOGU\AppData\Local\Programs\Python\Python311\Lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

